<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1171875
Current benign train loss: 2.360170841217041

Current batch: 100
Current benign train accuracy: 0.296875
Current benign train loss: 1.8470230102539062

Current batch: 200
Current benign train accuracy: 0.3515625
Current benign train loss: 1.6660887002944946

Current batch: 300
Current benign train accuracy: 0.3984375
Current benign train loss: 1.6940929889678955

Total benign train accuarcy: 33.554
Total benign train loss: 721.8320825099945

[ Test epoch: 0 ]

Test accuarcy: 40.64
Test average loss: 0.01662958071231842
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.46875
Current benign train loss: 1.6628693342208862

Current batch: 100
Current benign train accuracy: 0.4296875
Current benign train loss: 1.3959999084472656

Current batch: 200
Current benign train accuracy: 0.46875
Current benign train loss: 1.3807048797607422

Current batch: 300
Current benign train accurac


Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3228028118610382

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.2731390595436096

Total benign train accuarcy: 87.468
Total benign train loss: 143.15873017907143

[ Test epoch: 13 ]

Test accuarcy: 79.87
Test average loss: 0.006070816969871521
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.890625
Current benign train loss: 0.3537963032722473

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.174120232462883

Current batch: 200
Current benign train accuracy: 0.8515625
Current benign train loss: 0.5042714476585388

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.35935649275779724

Total benign train accuarcy: 88.19
Total benign train loss: 134.44040310382843

[ Test epoch: 14 ]

Test accuarcy: 81.85
Test average loss: 0.0055530628889799115
Model Save


Total benign train accuarcy: 91.496
Total benign train loss: 95.84548166394234

[ Test epoch: 26 ]

Test accuarcy: 86.1
Test average loss: 0.0040035804748535155
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.23580901324748993

Current batch: 100
Current benign train accuracy: 0.875
Current benign train loss: 0.372760146856308

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2770439684391022

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.24636922776699066

Total benign train accuarcy: 91.702
Total benign train loss: 93.44763531535864

[ Test epoch: 27 ]

Test accuarcy: 86.21
Test average loss: 0.004256710284948349
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2074543982744217

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train


Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.2624911665916443

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.25044286251068115

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1250627338886261

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1514568328857422

Total benign train accuarcy: 92.806
Total benign train loss: 80.04692854359746

[ Test epoch: 40 ]

Test accuarcy: 86.76
Test average loss: 0.004221188041567802
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12739761173725128

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.20668527483940125

Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.15063518285751343

Current batch: 300
Current benign train accuracy: 0.890625


Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.045735083520412445

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.04303480312228203

Total benign train accuarcy: 98.722
Total benign train loss: 15.40409644274041

[ Test epoch: 53 ]

Test accuarcy: 94.05
Test average loss: 0.0020274925511330364
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.016451731324195862

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.039372269064188004

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.021241065114736557

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.050303541123867035

Total benign train accuarcy: 98.97
Total benign train loss: 12.913918981328607

[ Test epoch: 54 ]

Test accuarcy: 93.98
Test average loss: 0.00208104488477110


Total benign train accuarcy: 99.68
Total benign train loss: 4.3675822634832

[ Test epoch: 66 ]

Test accuarcy: 93.9
Test average loss: 0.002358284318819642
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.019439097493886948

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.025608764961361885

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.004822699818760157

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0041029248386621475

Total benign train accuarcy: 99.766
Total benign train loss: 3.632731690420769

[ Test epoch: 67 ]

Test accuarcy: 94.2
Test average loss: 0.0023420155610889195
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.02867363765835762

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006059765350073576

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.011570571921765804

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.001692752237431705

Total benign train accuarcy: 99.882
Total benign train loss: 2.073586750746472

[ Test epoch: 80 ]

Test accuarcy: 93.98
Test average loss: 0.002451171316578984
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.004998610820621252

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.011245932430028915

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.001113425474613905

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.003885730402544141

Total benign train accuarcy: 99.884
Total benign train loss: 2.1090909953636583

[ T


Total benign train accuarcy: 99.966
Total benign train loss: 1.0745443194755353

[ Test epoch: 93 ]

Test accuarcy: 94.4
Test average loss: 0.002355556133389473
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.002723648911342025

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0009478553547523916

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0011641753371804953

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0038492176681756973

Total benign train accuarcy: 99.966
Total benign train loss: 0.9820212842314504

[ Test epoch: 94 ]

Test accuarcy: 94.4
Test average loss: 0.0023377304991707205
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0006796027882955968

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00074924